In [36]:
import pandas as pd
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from sklearn.metrics import recall_score, roc_auc_score, roc_curve, confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import cross_val_score
import statsmodels.api as sm
import collections as c
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', 2000)
pd.set_option('display.max_rows', 2000)

In [2]:
# model = pickle.load(open('logistic_regression_completion.p', 'rb')) # load the model

In [3]:
model = LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=200, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start='False')

In [4]:
def scale_subset(df, columns):
    '''
    Use sklearn StandardScalar to scale only numeric columns.

    Parameters:
    ----------
    input {dataframe, list}: dataframe containing mixed feature variable types, list of names of numeric feature columns
    output: {dataframe}: dataframe with numeric features scaled and categorical features unchanged

    '''
    scalar = StandardScaler()
    numeric = df[columns]
    categorical = df.drop(columns, axis = 1)
    scalar.fit(numeric)
    num_scaled = pd.DataFrame(scalar.transform(numeric))
    num_scaled.rename(columns = dict(zip(num_scaled.columns, numeric_cols)), inplace = True)
    return pd.concat([num_scaled, categorical], axis = 1)


In [5]:
X_train = pd.read_csv('../data/processed/X_train.csv')
y_train = pd.read_csv('../data/processed/y_train.csv')
# y_train = y_train['module_not_completed']
X_test = pd.read_csv('../data/processed/X_test.csv')

y_test = pd.read_csv('../data/processed/y_test.csv')
# y_test = y_test['module_not_completed']

numeric_cols = ['num_of_prev_attempts', 'studied_credits',
'clicks_per_day', 'pct_days_vle_accessed','max_clicks_one_day',
'first_date_vle_accessed', 'avg_score', 'avg_days_sub_early', 'days_early_first_assessment',
'score_first_assessment']

# fill and scale
X_train.fillna(value = 0, inplace = True)
X_train = scale_subset(X_train, numeric_cols)
X_test.fillna(value = 0, inplace = True)
X_test = scale_subset(X_test, numeric_cols)



/Users/jeremymiller/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/jeremymiller/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  from ipykernel import kernelapp as app
/Users/jeremymiller/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/jeremymiller/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  from ipykernel import kernelapp as app


In [6]:
# drop student who did not complete
withdrawls = []
for index, row in y_train[y_train['final_result_num'] == 0].iterrows():
    withdrawls.append(index)

In [7]:
# drop features with collinearity (need VIF)
X_train.drop(withdrawls, inplace=True)
X_train.drop(['avg_score', 'avg_days_sub_early', 'clicks_per_day'], axis=1, inplace=True)
X_test.drop(['avg_score', 'avg_days_sub_early', 'clicks_per_day'], axis=1, inplace=True)
y_train.drop(withdrawls, inplace=True)

In [10]:
y_train = y_train['final_result_num']
y_test = y_test['final_result_num']

In [11]:
model.fit(X_train, y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=200, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start='False')

In [12]:
predictions = model.predict(X_test)

In [13]:
recall = recall_score(y_test, predictions)
recall

ValueError: Target is multiclass but average='binary'. Please choose another average setting.

In [14]:
roc_auc = roc_auc_score(y_test, predictions)
roc_auc

ValueError: multiclass format is not supported

In [ ]:
probas = model.predict_proba(X_test)[:, :1]
tprs, fprs, thresh = roc_curve(y_test, probas)

In [ ]:
plt.figure(figsize=(12,10))
plt.plot(fprs, tprs, 
    label='ROC Curve AUC: {}\n\nRecall: {}'.format(roc_auc, recall),
    color='red')
plt.plot([0,1],[0,1], 'k:')
plt.legend()
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("Logistic Regression: Did the Student Complete the Course?")
plt.show()

In [ ]:
# pd.DataFrame(data={'fprs': fprs, 'tprs': tprs, 'Thresholds': thresh})

# Which features are most important?

In [ ]:
sm.stats.outliers_influence.va

In [22]:
y_train.shape

(15031,)

In [21]:
y_test.shape

(10768,)

In [37]:
abs_coef = list(np.abs(model.coef_.ravel()))
features = list(X_test.columns)

In [38]:
coef_dict = c.OrderedDict((zip(abs_coef, features)))

In [39]:
sorted(coef_dict.items(), reverse=True)

[(1.8468252497035902, 'code_module_FFF'),
 (1.6180785804110889, 'pct_days_vle_accessed'),
 (0.8852038617645583, 'score_first_assessment'),
 (0.7829204724977672, 'code_presentation_2014J'),
 (0.7029692997658055, 'code_module_BBB'),
 (0.6568863010720503, 'code_module_GGG'),
 (0.6434678061843189, 'imd_band_nan'),
 (0.6237095655697216, 'code_module_DDD'),
 (0.5768394331578773, 'highest_education_Lower Than A Level'),
 (0.5652509288070615, 'code_presentation_2013J'),
 (0.5438471411822636, 'code_presentation_2014B'),
 (0.4878665806300258, 'highest_education_No Formal quals'),
 (0.48359211922534356, 'imd_band_90-100%'),
 (0.45932480644946855, 'highest_education_Post Graduate Qualification'),
 (0.4449587317607533, 'imd_band_80-90%'),
 (0.36644832387034826, 'imd_band_60-70%'),
 (0.3189647882559601, 'days_early_first_assessment'),
 (0.31596802107234845, 'region_Scotland'),
 (0.3126435785006441, 'imd_band_30-40%'),
 (0.3120638799384648, 'region_Ireland'),
 (0.2872149701105499, 'imd_band_40-50%'),

In [15]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      3330
           1       0.32      0.65      0.42      2384
           2       0.62      0.87      0.72      4046
           3       0.59      0.13      0.21      1008

   micro avg       0.48      0.48      0.48     10768
   macro avg       0.38      0.41      0.34     10768
weighted avg       0.36      0.48      0.39     10768



/Users/jeremymiller/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [18]:
accuracy_score(y_test, predictions)

0.4812407132243685

In [16]:
print(confusion_matrix(y_test, predictions))

[[   0 2871  456    3]
 [   0 1540  841    3]
 [   0  450 3511   85]
 [   0   13  864  131]]


In [ ]:
# pickle.dump(model, open('logistic_regression_completion.p', 'wb')) # pickle the model

In [32]:
cor = X_train.corr().abs()

In [33]:
s = c.unstack()

In [34]:
so = s.sort_values(kind="quicksort", ascending=False)

In [35]:
so[::2]

disability_Y                                   disability_Y                                     1.000000
region_South Region                            region_South Region                              1.000000
region_Wales                                   region_Wales                                     1.000000
region_Yorkshire Region                        region_Yorkshire Region                          1.000000
highest_education_Lower Than A Level           highest_education_Lower Than A Level             1.000000
highest_education_Post Graduate Qualification  highest_education_Post Graduate Qualification    1.000000
imd_band_20-30%                                imd_band_20-30%                                  1.000000
imd_band_40-50%                                imd_band_40-50%                                  1.000000
imd_band_60-70%                                imd_band_60-70%                                  1.000000
imd_band_80-90%                                imd_band